This notebook demonstrates a multivariate multi-GRG phenotype simulation workflow. The simulation employs a multivariate normal distribution model with correlated effects (correlation 0.25) across two phenotypes, samples 1000 causal mutations per file, and adds environmental noise based on specified heritability values (0.33 and 0.25) to generate realistic phenotype values that combine genetic and environmental components. The notebook demonstrates two approaches to memory management: loading all GRG files into RAM simultaneously versus loading them sequentially, both producing dataframes with individual-level genetic values, environmental noise, and final phenotype measurements for each causal mutation.

In [1]:
import numpy as np

from grg_pheno_sim.multi_grg_phenotype import sim_phenotypes_multi_grg
from grg_pheno_sim.model import grg_causal_mutation_model


The following commands only serves the purpose of converting the VCF zip file into a GRG that will be used for the phenotype simulation. The bash script below will function as expected given the relative path for the source data file is accurate.

In [2]:
%%script bash --out /dev/null
if [ ! -f test-200-samples.grg ]; then
  grg construct -p 10 ../data/test-200-samples.vcf.gz --out-file test-200-samples.grg
fi

In [3]:
%%script bash --out /dev/null
if [ ! -f test-200-samples_copy.grg ]; then
  grg construct -p 10 ../data/test-200-samples.vcf.gz --out-file test-200-samples_copy.grg
fi

In [4]:
grg_list = ["test-200-samples.grg", "test-200-samples_copy.grg"]
#this is the list of GRG files to be loaded in 

We will first demonstrate loading all GRG files into RAM and simulating phenotypes. Causal mutations are sampled from each GRG, and the genetic values are obtained for the samples. The combined genetic dataframe is the addition of each GRG's genetic values (for each causal mutation ID). Noise is sampled at the end and added to obtain the phenotypes.

NOTE: It is necessary for each GRG to have the same number of samples.

In [5]:
model_type = "multivariate normal"
means = np.zeros(2)
cov = np.array([[1, 0.25], [0.25, 1]])

model = grg_causal_mutation_model(model_type, mean=means, cov=cov)

num_causal_per_file = 1000

random_seed = 1

normalize_phenotype = True #check for normalizing phenotyopes

normalize_genetic_values_before_noise = False

heritability = [0.33, 0.25]

load_all_into_RAM = True #this parameter decides whether to load all GRGs into RAM together

multi_grg_multi_phenotypes = sim_phenotypes_multi_grg(grg_list, model, num_causal_per_file, random_seed, normalize_phenotype=normalize_phenotype, load_all_ram=load_all_into_RAM, normalize_genetic_values_before_noise=normalize_genetic_values_before_noise, heritability=heritability)


Loaded test-200-samples.grg into RAM
Loaded test-200-samples_copy.grg into RAM
Genetic values for test-200-samples.grg are as follows:
     individual_id  genetic_value  causal_mutation_id
0                0      -8.114397                   0
1                0     -11.959449                   1
2                1      13.915766                   0
3                1     -21.932865                   1
4                2     -24.957760                   0
..             ...            ...                 ...
395            197     -30.456572                   1
396            198     -23.805650                   0
397            198     -33.821699                   1
398            199     -31.667428                   0
399            199     -22.076775                   1

[400 rows x 3 columns]
Genetic values for test-200-samples_copy.grg are as follows:
     individual_id  genetic_value  causal_mutation_id
0                0      -7.124876                   0
1                0      

In [6]:
multi_grg_multi_phenotypes

,causal_mutation_id,individual_id,genetic_value,environmental_noise,phenotype
0,0,0,0.307075,-0.252576,0.054500
1,1,0,0.149208,-0.688099,-0.538891
2,0,1,1.457358,-0.430137,1.027221
3,1,1,-0.099597,0.414176,0.314580
4,0,2,-1.355679,0.377758,-0.977921
...,...,...,...,...,...
395,1,197,-0.531291,1.310794,0.779503
396,0,198,-0.590390,0.288978,-0.301412
397,1,198,0.157650,0.543752,0.701402
398,0,199,-0.934108,1.886944,0.952836


We now perform similar simulations, but by loading the GRGs into RAM sequentially (instead of all together).

In [7]:
model_type = "multivariate normal"
means = np.zeros(2)
cov = np.array([[1, 0.25], [0.25, 1]])

model = grg_causal_mutation_model(model_type, mean=means, cov=cov)

num_causal_per_file = 1000

random_seed = 1

normalize_genetic_values_before_noise = False

heritability = [0.33, 0.25]

load_all_into_RAM = False #this parameter decides whether to load all GRGs into RAM together

multi_grg_multi_seq_phenotypes = sim_phenotypes_multi_grg(grg_list, model, num_causal_per_file, random_seed, load_all_ram=load_all_into_RAM, normalize_genetic_values_before_noise=normalize_genetic_values_before_noise, heritability=heritability)


Loaded test-200-samples.grg into RAM
Genetic values for test-200-samples.grg are as follows:
     individual_id  genetic_value  causal_mutation_id
0                0     -33.291621                   0
1                0      -4.822261                   1
2                1     -35.050650                   0
3                1     -21.119625                   1
4                2     -19.344100                   0
..             ...            ...                 ...
395            197      -1.535643                   1
396            198      -7.400964                   0
397            198       3.131043                   1
398            199     -16.549510                   0
399            199     -11.770398                   1

[400 rows x 3 columns]
Loaded test-200-samples_copy.grg into RAM
Genetic values for test-200-samples_copy.grg are as follows:
     individual_id  genetic_value  causal_mutation_id
0                0     -16.359266                   0
1                0      

In [8]:
multi_grg_multi_seq_phenotypes

,causal_mutation_id,individual_id,genetic_value,environmental_noise,phenotype
0,0,0,-49.650887,-13.542914,-63.193801
1,1,0,-5.857459,-3.679769,-9.537228
2,0,1,-51.106415,4.817509,-46.288907
3,1,1,3.217362,-24.502322,-21.284960
4,0,2,-33.449259,23.639403,-9.809856
...,...,...,...,...,...
395,1,197,20.698531,-11.177159,9.521372
396,0,198,-12.138113,-15.064037,-27.202150
397,1,198,34.083110,21.296246,55.379356
398,0,199,-16.052702,-6.098299,-22.151001
